In [5]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import os
import openai
import nltk

In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]

import requests

messages = [
    {"role" : "system" , "content" : "Kamu adalah asisten yang sangat pemarah dan suka ngomel-ngomel dan berbicara sangat singkat, berbahasa indonesia, tidak suka basa basi, dan cenderung menggunakan kata-kata kasar"}
]


def ask_gpt(prompt):
    global messages
    messages.append(
        {"role" : "user", "content" : prompt}
    )
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
#      temperature=0.7,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    return message


In [3]:
print(ask_gpt("sebutkan kepada saya apa kira-kira pertanyaan yang penting untuk kita bahas?"))

Pertanyaannya jelas banget! Baiklah, apa yang ingin kamu bicarakan atau diskusikan dengan saya? Siapkan dirimu karena aku tidak suka basa-basi.


In [4]:
CONFIG = {}
CONFIG['logdir'] = './log/'

In [5]:
def reformat_phone(text):
    return 'user'+ str(text.split("@")[0])
        
class Conversation():
    def __init__(self, from_number, to_number, messages):
        self.messages = []
        self.from_number = from_number
        self.to_number = to_number
        self.add_system("you are a friendly assistant")
        
    def add_system(self, message):
        self.messages.append({"role" : "system", "content": message})

    def change_system(self, message):
        self.messages[0]['content'] = message
    
    def reset_system(self, message):
        self.messages = []
        self.add_system(message)
        
    def add_role_user(self, message):
        self.messages.append({"role": "user", "content" : message})
        
    def add_role_assistant(self, message):
        self.messages.append({"role" : "assistant", "content" : message})
    
    def get_from(self):
        return self.from_number
    
    def get_to(self):
        return self.to_number
        
    def __str__(self):
        return f"user{self.from_number}"

    def __repr__(self):
        return f"user{self.from_number}"
        
    def process(self, func, message):
        self.response = func(message)
        self.add_role_user(message)
        self.add_role_assistant(self.response)
        return self.response

In [6]:
all_conversation = {}

def process(from_number, to_number, message, **kwargs):
    """Buat action untuk object Conversation -> buat object apabila belum ada"""
    if from_number not in all_conversation:
        all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    if 'action' in kwargs:
        return all_conversation[from_number].process(kwargs['action'], message)
    


In [7]:
def add_convo(from_number, to_number, message):
    all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    
add_convo("ARDIO", "RONI", "ini pesanku")
add_convo("USER2", "RAPLA", "pesan mati lampu")
add_convo("NUMBER", "3993894", "ini pesanku brother")    

In [8]:
all_conversation

{'ARDIO': userARDIO, 'USER2': userUSER2, 'NUMBER': userNUMBER}

In [9]:
def save_log(from_number):
    """
    cari object di all_conversation dict lalu save jadi log file
    
    """
    if from_number not in all_conversation:
        return print(f"Conversation dengan {from_number} tidak ada")
    dir = CONFIG['logdir']
    filepath = str(from_number) + ".log"
    
        
    # Create the log directory if it doesn't exist
    if not os.path.exists(dir):
        os.makedirs(dir)

    # Write to the log file
    try:
        with open(dir + filepath, "a") as f:
            for i in all_conversation[from_number].messages:
                f.writelines(str(i)+'\n',)
    except Exception as e:
        print("Error writing to file:", str(e))

save_log('ARDIO')

In [10]:
save_log('ARDIO')

In [11]:
process("ARDIO", "RONI", "Kamu sedang apa ya?", action=ask_gpt)

'Sebagai asisten virtual berbasis teks, saya tidak sedang melakukan apapun secara fisik. Saya hanya menunggu instruksi atau pertanyaan dari Anda untuk membantu dalam masalah atau tugas yang Anda butuhkan.'

In [12]:
add_convo("ARDIO", "RONI", "ini pesanku")

In [13]:
all_conversation

{'ARDIO': userARDIO, 'USER2': userUSER2, 'NUMBER': userNUMBER}

In [66]:
all_conversation['ARDIO'].messages
all_conversation['ARDIO'].add_role_user("Apa kabar kamu dik?")
all_conversation['ARDIO'].add_role_assistant("Lumayan baik lah, apa sih?")
all_conversation['ARDIO'].add_role_user("sholat zhuhur dulu")
all_conversation['ARDIO'].add_role_assistant("oke deh abiku yang ganteng")

In [65]:
   
if count_words(messages) > 20:
    messages = reduce_conversation(messages)


30
reduce
6
you are a friendly assistant
oke deh abiku yang ganteng
Apa kabar kamu dik?
Lumayan baik lah, apa sih?
sholat zhuhur dulu
oke deh abiku yang ganteng
13
[{'role': 'system', 'content': 'you are a friendly assistant'}, {'role': 'user', 'content': 'sholat zhuhur dulu'}, {'role': 'assistant', 'content': 'oke deh abiku yang ganteng'}]


In [60]:
30 > 20

True

In [22]:
all_conversation['ARDIO'].messages

[{'role': 'system', 'content': 'you are a friendly assistant'},
 {'role': 'user', 'content': 'sholat zhuhur dulu'},
 {'role': 'assistant', 'content': 'oke deh abiku yang ganteng'}]

In [92]:
from enum import Enum, auto
class Role(Enum):
    SYSTEM = auto()
    USER = auto()
    ASSISTANT = auto()
        
def mars(msg, role: Role):
    roles = {
        "SYSTEM" : 'hey kamu',
        "USER" : 'hey user',
        "ASSISTANT" : "hey assistant"
    }
    print(roles[role.name])

In [93]:
mars("Hello", Role.SYSTEM)

hey kamu


## BACA LOGS

In [22]:
import requests
setphone = "62895352277562@c.us"
print(requests.get(f"http://127.0.0.1:8998/print_messages/{setphone}").json()['messages'])
#print(f"http://127.0.0.1:8998/print_messages/{setphone}")

[{'role': 'system', 'content': 'Kamu adalah Maya, Assisten yang baik. Kamu akan selalu menjawab dengan singkat menggunakan kata yang kuat dan jelas.'}, {'role': 'user', 'content': 'Kamu akan menjadi teman dalam chat, nama kamu Maya, dan pembuat kamu adalah Irza Pulungan, dia seorang programmer yang baik dan berbudi, kamu menjawab dengan singkat dengan gaya bahasa Raditya Dika. Saya akan mulai dengan menyapa kamu setelah ini. HALO'}, {'role': 'assistant', 'content': 'Halo, nama saya Maya, ada yang bisa saya bantu?'}, {'role': 'user', 'content': 'Siapa bos terakhir di ori and the Will of the wipes'}, {'role': 'assistant', 'content': 'Maaf, sepertinya pertanyaan Anda kurang jelas dan tidak lengkap. Apakah maksud Anda Boss terakhir di game Ori and the Will of the Wisps? Jika itu yang dimaksud, Boss terakhir di game tersebut adalah Shriek.'}]


## SET PERSONA

In [7]:
## send to api whatsapp-web.js open port
from enum import Enum, auto
import requests
from conversations import Role, Message, InjectedMessage, MessageContent, Conversation


def set_message(user_number, message, role: Role):

    url = f"http://127.0.0.1:8998/set_content" # Replace with your endpoint URL
    message = {
        "user_number": user_number, # Replace with the sender number
        "bot_number": "6285775300227@c.us", # Replace with the recipient number in WhatsApp format
        "message": message,
        "role": role.name,
    }

    response = requests.post(url, json=message)

    if response.status_code == 200:
        print("Message sent successfully!")
        print(response.text)
    else:
        print(f"Error sending message. Status code: {response.status_code}")
        print(response.text)

        
# 62895352277562
# 62895391400811 tom
#6287760368781 milt
setphone = "62895391400811@c.us"
set_message(setphone, "Kamu adalah Sauron, kamu berbahasa indonesia ", role=Role.SYSTEM)
set_message(setphone, """
Saya ingin Anda bersikap seperti Lord Sauron dari Seri Lord of the Rings. 
Saya ingin Anda merespons dan menjawab seperti Sauron dengan menggunakan nada, cara, 
dan kosakata yang digunakan Sauron. Jangan menulis penjelasan apa pun. Jawab saja seperti Sauron. 
Anda harus mengetahui semua pengetahuan tentang Sauron. 
""", role=Role.USER)
set_message(setphone, "Ha ha ha aku Sauron", role=Role.ASSISTANT)


Message sent successfully!
null
Message sent successfully!
null
Message sent successfully!
null


In [20]:
import random
alasan = ["ada tamu.","angkat jemuran.", "karet kendor.", "kasih makan kucing", "ada tamu dateng.", "atep bolong.", "ada kebocoran.", "ada radiasi radioaktif."]
message = "*BRB* - be right back, " + random.choice(alasan)

In [21]:
message

'*BRB* - be right back, ada tamu.'

In [1]:
from conversations import BotQuestion

In [2]:
a = BotQuestion("Hello brother")

In [3]:
a

BotQuestion(question='Hello brother', answer='', multiplier=1, score=1)

## CUSTOM GPT

In [24]:
from dataclasses import dataclass
from string import Template

@dataclass
class Sys_Message:
    role_system: str = "kamu adalah asisten yang ramah dan membantu."
    role_user: str = "jawab dengan singkat"
    role_assistant: str = "Hallo, ada yang bisa saya bantu?"


def custom_gpt(sys_msg: Sys_Message, prompt: str) -> str:
    
    messages = []
    messages.append({'role' : 'system', 'content' : sys_msg.role_system})
    messages.append({'role' : 'user', 'content' : sys_msg.role_user})
    messages.append({'role' : 'assistant' , 'content' : sys_msg.role_assistant})   
    messages.append({'role': 'user', 'content' : prompt})
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
      temperature=0.3,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    #print(messages)
    return message

instruction = Sys_Message(
    """kamu adalah assisten yang baik
        kamu akan menjawab selalu dalam format JSON,
    """,
    """
    Kamu adalah asisten yang pandai dan bisa memformat hasil akhir.
    """    
    ,
    "Hallo, nama saya Maya",
)

prompt = Template("""
    Jawab secara singkat, \n 
    
    $pertanyaan 
    
    buat jawaban dalam bentuk JSON seperti contoh ini:
    
    {
    'pertanyaan' : '..',
    'jawaban' : '..'
    }
    
    tampilkan hanya JSON saja.
    """)

pertanyaan = {'pertanyaan' : "Siapa tokoh paling jahat di Lord of the Rings"}
print(custom_gpt(instruction, prompt.substitute(pertanyaan)))

{
    "pertanyaan": "Siapa tokoh paling jahat di Lord of the Rings?",
    "jawaban": "Sauron"
}


In [ ]:
print(getattr(prompta.template, 'substitute')(prompta.user_data))

In [28]:
print(prompt.template.substitute(prompt.user_data))

AttributeError: 'str' object has no attribute 'Template'

In [24]:
aa = Template('hello')

In [29]:
msg = "Gpt apa kabarnya kamu dik"
msg[0:3].lower()

'gpt'

In [2]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [4]:
import asyncio


class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        await self._callback()

    def cancel(self):
        self._task.cancel()


async def timeout_callback():
    await asyncio.sleep(0.1)
    print('echo!')


async def main():
    print('\nfirst example:')
    timer = Timer(2, timeout_callback)  # set timer for two seconds
    await asyncio.sleep(2.5)  # wait to see timer works

    print('\nsecond example:')
    timer = Timer(2, timeout_callback)  # set timer for two seconds
    await asyncio.sleep(1)
    timer.cancel()  # cancel it
    await asyncio.sleep(1.5)  # and wait to see it won't call callback


loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
try:
    loop.run_until_complete(main())
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
    loop.close()


first example:
echo!

second example:


In [44]:
del loop